# KF Homework: Constructing KF Parameters

To find the questions, search for triple-bangs "**!!!**"



## The Coupled Spring Oscillators

### Constructing a discrete time motion model

Consider a mass attached to a wall on the left by a spring, a second mass attached to a wall on the right by a spring, and a spring running between the two masses; for example, see [this picture](https://commons.wikimedia.org/wiki/File:Coupled_Harmonic_Oscillator.svg).

Ignoring friction, we can model this system with the following ordinary differential equations (ODEs):
$$
  \begin{aligned}
    \ddot x_1(t) &= -(k_1 + k_m) x_1(t) + k_m x_2(t) \\
    \ddot x_2(t) &= k_m x_1(t) - (k_2 + k_m) x_2(t)
  \end{aligned}
$$
where
* $x_1(t)$ and $x_2(t)$ are the displacement of mass 1 and mass 2 respectively from their equilibrium locations as a function of time.
* $\dot x(t)$ is shorthand for the first derivative of $x(t)$ with respect to time $t$, and $\ddot x(t)$ is shorthand for the second derivative of $x(t)$ with respect to time $t$.
* We have assumed that the masses have unit mass to normalize the equations.
* $k_1$, $k_2$ and $k_m$ are the spring constants of the springs attaching mass 1 to the wall, mass 2 to the wall and mass 1 to mass 2 respectively.

We can convert these two second order ODEs into four first order ODEs by adding two more states $v_1(t)$ and $v_2(t)$ which are the velocities of mass 1 and mass 2 respectively:
$$
  \begin{aligned}
    \dot x_1(t) &= v_1(t) \\
    \dot v_1(t) &= -(k_1 + k_m) x_1(t) + k_m x_2(t)\\
    \dot x_2(t) &= v_2(t) \\
    \dot v_2(t) &= k_m x_1(t) - (k_2 + k_m) x_2(t)
  \end{aligned}
$$

We can then convert this ODE into a discrete time approximation by using the [Forward Euler](https://en.wikipedia.org/wiki/Euler_method) method:
$$
  \begin{aligned}
    x_1(t+h) &= x_1(t) + h v_1(t) \\
    v_1(t+h) &= v_1(t) - h(k_1 + k_m) x_1(t) + h k_m x_2(t) \\
    x_2(t+h) &= x_2(t) + h v_2(t) \\
    v_2(t+h) &= v_2(t) + h k_m x_1(t) - h (k_2 + k_m) x_2(t)
  \end{aligned}
$$
where $h$ is the time step size.

Of course, such an equation is inevitably an inaccurate model of the real world since we have ignored effects like friction or errors in the model parameters (the masses of the two oscillators, $k_1$, $k_2$ and $k_m$ in this case).  We often account for such inaccuracies by adding a noise term to our model.

Following the notation from these [Kalman filter slides](http://probabilistic-robotics.informatik.uni-freiburg.de/ppt/kalman.ppt), define a state vector $\bar x = \begin{bmatrix} x_1 & v_1 & x_2 & v_2 \end{bmatrix}^T$.  After adding noise, the motion model above can be written as 
$$
  \bar x(t+1) = A \bar x(t) + B u(t) + h \epsilon
$$
Assume that $\epsilon$ is drawn from a normal distribution with mean 0 and variance $\sigma^2_m$.

### Constructing the KF motion parameters

**!!!** Write out the KF parameters for this motion model (in mathematical notation).  These parameters will depend on $k_1$, $k_2$, $k_m$ and $h$.  Each parameter should receive a bullet point which describes the role of that parameter in the model; for example, "motion noise".

You may write an $n \times n$ identity matrix as $I_n$.

* State transition matrix:
  $$
    A = 
      \begin{bmatrix}
        1 & h & 0 & 0 \\
        -h(k_1 + k_m) & 1 & hk_m & 0 \\
        0 & 0 & 1 & h \\
        hk_m & 0 & -h(k_2 + k_m) & 1
      \end{bmatrix}
  $$
* Control matrix:
  $$
    B =
      \begin{bmatrix}
        0 & 0 \\
        0 & 0 \\
        h & 0 \\
        0 & h
      \end{bmatrix}
  $$

### Constructing a sensing model and KF sensing parameters

**!!!** Assume that we can sense $x_1$ and $x_2$ with additive noise that is Gaussian with mean zero and variance $\sigma^2_s$.  Write out the KF parameters for this sensing model (in mathematical notation).  These parameters will depend on $\sigma^2_s$.

* Observation matrix:
  $$
    C =
      \begin{bmatrix}
        1 & 0 & 0 & 0\\
        0 & 0 & 1 & 0\\
      \end{bmatrix}
  $$

* Gaussian noise:
  $\delta(t) \sim N(0, \sigma^2_s)$
  
* Sensing model:
  $$
    z(t) = C x(t) + \delta(t)
  $$

### Implementing the KF

**!!!** Implement a KF for the coupled oscillators in the code block below.
* Concrete values are provided for the problem parameters.
* You will need to simulate the system to provide sensor data.
  * So you will track two things: The actual state, and the KF estimate of the state.
  * I have provided the code to track the actual state (stored in variable `x`).
  * At each step, you will update both according to the motion model (although the KF estimate will include noise).
  * At each step, your sensor reading will be the actual state.  In other words, the sensor turns out to be perfect even though we have assumed it is noisy.
* Assume that your actual initial condition has mass 1 offset by +1 unit from its equilibrium location, mass 2 at its equilibrium location, and both masses motionless.
* Assume that your initial estimated state is both masses at their equilibrium locations and motionless, but your initial estimate admits to large uncertainty (variance $\sigma^2_i = 10$).
* Execute your motion update (prediction) before your sensing (correction).


In [2]:
import numpy as np

# Following the recommendation of NumPy, my solution with use np.array() rather than np.matrix(),
# and then use the "@" operator for matrix multiplication (since "*" is elementwise multiplication).

# Problem parameters.
k1 = 1.0
k2 = 1.0
km = 1.0
sigma2m = 1.0e-2
sigma2s = 2.0e-1
sigma2i = 10.0

# Do you want to see output every timestep?  (Final output is always produced.)
viz = False

# Timestep.
h = 0.1
# Maximum simulation time.
tmax = 5.0

# Any additional parameters you might want to define.
I2 = np.identity(2)
I4 = np.identity(4)

# KF Model parameters
A = np.array([[1, h, 0, 0],
              [-h*(k1+km), 1, h*km, 0],
              [0, 0, 1, h],
              [h*km, 0, -h*(k2+km), 1]
             ])

# Initial conditions.
x0 = np.array((( 1, 0, 0, 0 ),)).transpose()
mu0 = np.zeros((4, 1))
Sigma0 = sigma2i * I4

# Initialize the changing variables.
x = x0
mu = mu0
Sigma = Sigma0
t = 0

# Loop until maximum time.
while((tmax - t) > 0.5 * h):

    # Time update.
    t = t + h

    # Motion update of actual state.
    x = A @ x
    
    # Motion update of estimated state.

    # Sensor reading.
    
    # Sensing update of estimated state.
    
    if viz:
        print("t = ", t)
        print("x^T = ", x.transpose())
        print("error^T = ", (x - mu).transpose())
        #print("Sigma = \n", Sigma)
        print("eig(Sigma) = ", np.sort(np.linalg.eigvals(Sigma)))
        print()

print("t = ", t)
print("x^T = ", x.transpose())
print("mu^T = ", mu.transpose())
print("error^T = ", (x - mu).transpose())
print("Sigma = \n", Sigma)
print("eig(Sigma) = ", np.sort(np.linalg.eigvals(Sigma)))


t =  4.999999999999998
x^T =  [[-0.51956609 -0.7439854   0.86292074  1.97959837]]
mu^T =  [[0. 0. 0. 0.]]
error^T =  [[-0.51956609 -0.7439854   0.86292074  1.97959837]]
Sigma = 
 [[10.  0.  0.  0.]
 [ 0. 10.  0.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0.  0. 10.]]
eig(Sigma) =  [10. 10. 10. 10.]


### The effect of losing a sensor

The eigenvalues of the covariance matrix $\Sigma$ provide the variances of the estimate in a coordinate system defined by the eigenvectors.  In particular, if all eigenvalues of $\Sigma$ are below some value $\sigma^2$, then you have localized at $mu$ with variance $\sigma^2$.  The eigenvalues of a matrix can be found with `np.linalg.eigvals()`.

**!!!** Answer the questions below using your code above.  Type your answer after the question mark.  Answers within a factor of two of the correct answer will receive full marks.

* What is the maximum variance of your uncertainty in the state of the system at $t = 10$?

* Now assume that we are no longer able to sense the position of the second mass.  To compensate, we will use a more accurate sensor on the first mass.  How much more accurate must the sensor be to achieve at least as good a level of localization at $t = 10$ with just the one sensor?  In other words, how much smaller must $\sigma^2_s$ be?

## The Kinematic Unicycle

An ODE model often used for slow moving ground robots (when inertia is small relative to the available actuation force) is the *kinematic unicycle*:
$$
  \begin{aligned}
    \dot x(t) &= v \cos \theta(t) \\
    \dot y(t) &= v \sin \theta(t) \\
    \dot \theta(t) &= \omega
  \end{aligned}
$$
where
* $x(t)$ and $y(t)$ are the position of the robot in the plane as a function of time.
* $\theta(t)$ is the heading as a function of time.
* $v$ and $\omega$ are the linear and angular velocities of the robot.  For now we will assume that these velocities are constant.


### Discrete time motion model

**!!!** Convert the ODE model shown above into a discrete time model using the Forward Euler method with time step $h$.



### KF for the unicycle?

**!!!** Briefly explain why you cannot use the (basic) Kalman Filter for this model.